In [2]:
import derinet.lexicon as dlex
import os
lexicon = dlex.Lexicon()
current_dir = os.getcwd()  # aktualni adresar
file_path = os.path.join(current_dir, "./derinet-2-3.tsv")  #sestaveni cesty
lexicon.load(file_path)

all_lemmas = {lex.lemma for lex in lexicon.iter_lexemes()}

test 26: ista musí být u ismus

In [ ]:
#tady to seřadit podle toho, do které if větve mi to zapadlo - když mám liberalista, tak ten je určitě pod liberalismus, ale 
#ženista nebude pod ženou

#odsud to vzít pro všechna slova, a pokud najdu víc, vyberu podle četnosti použití
#seradit podle toho, jak časté jsou ty nalezené 

def nejlepsi(kandidati):
    hodnota = 0
    nejvyssi = None 
    for i in kandidati:
        if i != None:
            slovo = lexicon.get_lexemes(i)[0]
            if slovo.misc['corpus_stats']['relative_frequency'] > hodnota:
                nejvyssi = slovo.lemma
                hodnota = slovo.misc['corpus_stats']['relative_frequency']    

    return nejvyssi 

def test(nove):
    kandidati = set()
    for i in "aeioyu":
        if nove + i in all_lemmas:
            slovicko = nove + i
            if slovicko in all_lemmas:
                kandidati.add(slovicko)
    return kandidati
                
def nova_slova(slovo):
    novoty = set()
    bez = "aeiouuy"
    s = "áéíóůúý"

    for i, znak in enumerate(slovo): #tohle míří na solista a sólo
        nove = None
        if znak in bez:
            index = bez.index(znak)
            nove = slovo[:i] + s[index] + slovo[i + 1:]
        elif znak in s:
            index = s.index(znak)
            nove = slovo[:i] + bez[index] + slovo[i+ 1:]
        
        if nove and nove in all_lemmas:
            novoty.add(nove)
    return novoty

def doplneni(slovo):
    kandidati = set()
    novoty = nova_slova(slovo)
    novoty.add(slovo)

    for i in novoty:
        vysledek = test(i)
        kandidati = kandidati | vysledek
        for j in vysledek:
            kandidati.add(j)

    if len(kandidati) > 1:
        reseni = nejlepsi(kandidati)
        return reseni
    
    elif len(kandidati) == 1:
        return kandidati.pop()
    
    else:
        return None

def kontrola(predek, lemma, f, koncovka):
    if koncovka == "nahoru":
        predek = predek[0].upper() + predek[1:]
    else:
        predek = predek + koncovka
    if predek in all_lemmas:
        if koncovka != "nahoru":
            f.write(f"{lemma}, {predek}\n")
            return None
        else:
            return predek
        
with open("test26.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma.endswith("ista") and lexeme.all_parents == []:
            predek = lexeme.lemma[:-4]

            if predek + "ismus" in all_lemmas:
                kontrola(predek, lexeme.lemma, f, "ismus")
                
            elif predek + "istika" in all_lemmas:
                kontrola(predek, lexeme.lemma, f, "istika")

            else:
                #sólista a solista
                kand = nova_slova(lexeme.lemma)
                if kand:
                    nej = nejlepsi(kand)
                    f.write(f"{lexeme.lemma}, {nej}\n")
                        
                else:
                    kandidati = []
                    if predek in all_lemmas: 
                        kandidati.append(predek)
                    kandidati.append(kontrola(predek, lexeme.lemma, f, "nahoru"))
                    kandidati.append(doplneni(predek + "i"))
                    kandidati.append(doplneni(predek))

                    nejvyssi = nejlepsi(kandidati)
                    if nejvyssi != None:
                        f.write(f"{lexeme.lemma}, {nejvyssi}\n")
                    else:
                        f.write(f"{lexeme.lemma} \n")
                


None
None
None
Ario
Arie
Ara
Are
Ari
None
Arie
Ari
None
None
None
Bate
Bata
None
None
Bata
Batty
Batto
None
Battie
Batto
None
None
None
Baša
Baše
Baš
Baš
None
Baše
None
None
None
None
None
Bova
Cali
Cale
Cal
Cal
None
Cale
Doly
Dolo
Doli
Dole
Dol
Dol
None
Doly
Elio
Elia
Elie
Eli
Elo
Ela
Ely
El
El
Elie
Eli
None
None
None
None
None
None
None
None
None
Gum
Gum
None
Gumy
None
None
None
Kala
Kaly
Kali
Kal
Kal
None
Kala
Kalla
Kalle
Kalli
None
None
Kalla
None
None
None
None
None
Koivu
Kopy
Kopa
Kop
Kop
None
Kopa
None
None
None
None
None
None
None
None
None
None
Kubie
Kuba
Paula
Pauli
Paulo
Paul
Paul
None
Paulo
None
None
Posa
None
None
None
None
None
None
None
None
Tura
None
None
Vo
Vlado
Vladi
Vlad
Vlad
None
Vlado
None
None
None
afiš
None
None
None
None
None
None
None
None
akcese
None
None
alternativa
None
None
None
None
None
None
None
None
anály
None
None
None
None
None
None
None
None
None
antiradar
None
None
None
None
None
None
None
None
None
ar
None
aria
ara
Arkan
None
None
None
artilerie
N

In [117]:
for i in lexicon.iter_lexemes():
    if i.lemma == "blogg":
        print(i.lemma)


test 27: ice (loděnice)

In [125]:
with open("test27.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma.endswith("ice") and lexeme.lemma[0].islower():
            if 'Gender' in lexeme.feats and lexeme.feats['Gender'] == 'Fem':
                predek = lexeme.lemma[:-3]
                    #malinove -> malina 
                if len(predek) > 2 and predek[-2] == "ě":
                    predek = predek[:-2]
                    znaky = "ďťň"
                    for i, znak in enumerate("dtn"):
                        if predek[-1] == znak:
                            predek = predek + znaky[i]
                        
                        if predek in all_lemmas:
                            f.write(f"{lexeme.lemma}, {predek} \n")
                        

                #f.write(f"{lexeme.lemma}\n")

test 28: houbaření -> houbařit (když je na konci -it, je to něco?)

test 29: houbař -> houba

test 30: houbařit, telefonovat, sportovat

test 31: AltaVista